In [ ]:
import pandas as pd

def detect_hard_braking(df):
    """
    Detect hard braking incidents based on the given criteria.
    """
    hard_braking_events = []

    # Get unique vehicle IDs
    unique_vehicles = df["Tracker ID"].unique()

    for vehicle_id in unique_vehicles:
        vehicle_data = df[df["Tracker ID"] == vehicle_id].sort_values(by="Frame").reset_index(drop=True)

        for i in range(len(vehicle_data) - 19):  # Ensuring a full 20-frame window
            # Define the sliding window
            window = vehicle_data.iloc[i : i + 20]

            # Get initial and final speed
            initial_speed = window.iloc[0]["Speed (mph)"]
            final_speed = window.iloc[-1]["Speed (mph)"]

            # Compute percentage drop in speed
            if initial_speed > 0:  # Avoid division by zero
                percentage_drop = ((initial_speed - final_speed) / initial_speed) * 100

                if percentage_drop > 60 and initial_speed > 15:
                    # Check the next 100 frames for speed fluctuations
                    post_window_data = vehicle_data.iloc[i + 20 : i + 120] if i + 120 < len(vehicle_data) else vehicle_data.iloc[i + 20 :]
                    if not post_window_data.empty:
                        max_post_speed = post_window_data["Speed (mph)"].max()

                        if max_post_speed <= final_speed + 5:  # Allow slight fluctuation
                            # Check speed trend in previous 20 frames
                            pre_window_data = vehicle_data.iloc[max(i - 20, 0) : i]

                            if not pre_window_data.empty:
                                min_pre_speed = pre_window_data["Speed (mph)"].min()
                                max_pre_speed = pre_window_data["Speed (mph)"].max()

                                if min_pre_speed >= initial_speed - 5 and max_pre_speed <= initial_speed + 5:
                                    # Log hard braking event
                                    hard_braking_events.append({
                                        "Tracker ID": vehicle_id,
                                        "Frame Start": window.iloc[0]["Frame"],
                                        "Frame End": window.iloc[-1]["Frame"],
                                        "Initial Speed (mph)": initial_speed,
                                        "Final Speed (mph)": final_speed,
                                        "Percentage Drop": percentage_drop,
                                        "Max Speed in Next 100 Frames": max_post_speed,
                                        "Min Speed in Previous 20 Frames": min_pre_speed,
                                        "Max Speed in Previous 20 Frames": max_pre_speed
                                    })

    # Convert results into a DataFrame
    return pd.DataFrame(hard_braking_events)

# Example usage
# df = pd.read_excel("your_data.xlsx")  # Load your dataset
# hard_braking_df = detect_hard_braking(df)
# hard_braking_df.to_excel("hard_braking_events.xlsx", index=False)  # Save results if needed


In [11]:
import pandas as pd

def detect_hard_braking(input_file, output_file):
    """
    Detects hard braking incidents from an Excel file and saves the results in a new Excel file.
    
    Parameters:
    - input_file: Path to the input Excel file containing trajectory data.
    - output_file: Path to save the detected hard braking incidents.
    """
    # Load the dataset
    df = pd.read_excel(input_file)

    hard_braking_events = []

    # Get unique vehicle IDs
    unique_vehicles = df["Tracker ID"].unique()

    for vehicle_id in unique_vehicles:
        vehicle_data = df[df["Tracker ID"] == vehicle_id].sort_values(by="Frame").reset_index(drop=True)

        for i in range(len(vehicle_data) - 19):  # Ensuring a full 20-frame window
            # Define the sliding window
            window = vehicle_data.iloc[i : i + 20]

            # Get initial and final speed
            initial_speed = window.iloc[0]["Speed (mph)"]
            final_speed = window.iloc[-1]["Speed (mph)"]

            # Compute percentage drop in speed
            if initial_speed > 0:  # Avoid division by zero
                percentage_drop = ((initial_speed - final_speed) / initial_speed) * 100

                if percentage_drop > 60 and initial_speed > 15:
                    # Check the next 100 frames for speed fluctuations
                    post_window_data = vehicle_data.iloc[i + 20 : i + 120] if i + 120 < len(vehicle_data) else vehicle_data.iloc[i + 20 :]
                    if not post_window_data.empty:
                        max_post_speed = post_window_data["Speed (mph)"].max()

                        if max_post_speed <= final_speed + 5:  # Allow slight fluctuation
                            # Check speed trend in previous 20 frames
                            pre_window_data = vehicle_data.iloc[max(i - 20, 0) : i]

                            if not pre_window_data.empty:
                                min_pre_speed = pre_window_data["Speed (mph)"].min()
                                max_pre_speed = pre_window_data["Speed (mph)"].max()

                                if min_pre_speed >= initial_speed - 5 and max_pre_speed <= initial_speed + 5:
                                    # Log hard braking event
                                    hard_braking_events.append({
                                        "Tracker ID": vehicle_id,
                                        "Frame Start": window.iloc[0]["Frame"],
                                        "Frame End": window.iloc[-1]["Frame"],
                                        "Initial Speed (mph)": initial_speed,
                                        "Final Speed (mph)": final_speed,
                                        "Percentage Drop": percentage_drop,
                                        "Max Speed in Next 100 Frames": max_post_speed,
                                        "Min Speed in Previous 20 Frames": min_pre_speed,
                                        "Max Speed in Previous 20 Frames": max_pre_speed
                                    })

    # Convert results into a DataFrame
    hard_braking_df = pd.DataFrame(hard_braking_events)

    # Save results to an Excel file
    hard_braking_df.to_excel(output_file, index=False)

    print(f"Hard braking incidents saved to {output_file}")

# Example usage
input_excel_file = "/Users/gideonowusu/Downloads/PhD 2028/Spring 2025/Coursework/5900/QC Sample Trajectory Data.xlsx"  # Replace with your input file
output_excel_file = "/Users/gideonowusu/Downloads/PhD 2028/Spring 2025/Coursework/5900/QC Sample Hard Braking Data.xlsx"  # Replace with desired output file name
detect_hard_braking(input_excel_file, output_excel_file)


Hard braking incidents saved to /Users/gideonowusu/Downloads/PhD 2028/Spring 2025/Coursework/5900/QC Sample Hard Braking Data.xlsx


In [ ]:
import pandas as pd

def detect_hard_braking(input_file, output_file):
    """
    Detects hard braking incidents from an Excel file and saves the results in a new Excel file,
    including the associated Line Tag.
    """
    # Load the dataset
    df = pd.read_excel(input_file)

    hard_braking_events = []

    # Get unique vehicle IDs
    unique_vehicles = df["Tracker ID"].unique()

    for vehicle_id in unique_vehicles:
        vehicle_data = df[df["Tracker ID"] == vehicle_id].sort_values(by="Frame").reset_index(drop=True)

        for i in range(len(vehicle_data) - 19):  # Ensuring a full 20-frame window
            # Define the sliding window
            window = vehicle_data.iloc[i : i + 20]

            # Get initial and final speed
            initial_speed = window.iloc[0]["Speed (mph)"]
            final_speed = window.iloc[-1]["Speed (mph)"]

            # Get the associated Line Tag (choosing the most frequent tag in the window)
            line_tag = window["Line Tag"].mode().iloc[0] if not window["Line Tag"].mode().empty else "Unknown"

            # Compute percentage drop in speed
            if initial_speed > 0:  # Avoid division by zero
                percentage_drop = ((initial_speed - final_speed) / initial_speed) * 100

                if percentage_drop > 60 and initial_speed > 15:
                    # Check the next 100 frames for speed fluctuations
                    post_window_data = vehicle_data.iloc[i + 20 : i + 120] if i + 120 < len(vehicle_data) else vehicle_data.iloc[i + 20 :]
                    if not post_window_data.empty:
                        max_post_speed = post_window_data["Speed (mph)"].max()

                        if max_post_speed <= final_speed + 5:  # Allow slight fluctuation
                            # Check speed trend in previous 20 frames
                            pre_window_data = vehicle_data.iloc[max(i - 20, 0) : i]

                            if not pre_window_data.empty:
                                min_pre_speed = pre_window_data["Speed (mph)"].min()
                                max_pre_speed = pre_window_data["Speed (mph)"].max()

                                if min_pre_speed >= initial_speed - 5 and max_pre_speed <= initial_speed + 5:
                                    # Log hard braking event
                                    hard_braking_events.append({
                                        "Tracker ID": vehicle_id,
                                        "Frame Start": window.iloc[0]["Frame"],
                                        "Frame End": window.iloc[-1]["Frame"],
                                        "Initial Speed (mph)": initial_speed,
                                        "Final Speed (mph)": final_speed,
                                        "Percentage Drop": percentage_drop,
                                        "Max Speed in Next 100 Frames": max_post_speed,
                                        "Min Speed in Previous 20 Frames": min_pre_speed,
                                        "Max Speed in Previous 20 Frames": max_pre_speed,
                                        "Line Tag": line_tag  # Include Line Tag
                                    })

    # Convert results into a DataFrame
    hard_braking_df = pd.DataFrame(hard_braking_events)

    # Save results to an Excel file
    hard_braking_df.to_excel(output_file, index=False)

    print(f"Hard braking incidents saved to {output_file}")

# Example usage
input_excel_file = "/Users/gideonowusu/Downloads/PhD 2028/Spring 2025/Coursework/5900/QC Sample Trajectory Data.xlsx"  # Replace with your input file
output_excel_file = "/Users/gideonowusu/Downloads/PhD 2028/Spring 2025/Coursework/5900/QC Sample Hard Braking Data.xlsx"  # Replace with desired output file name
detect_hard_braking(input_excel_file, output_excel_file)


In [12]:
!pip install xlsxwriter

!pip install -q supervision ultralytics

import os
import cv2
import numpy as np
import supervision as sv
from tqdm import tqdm
from ultralytics import YOLO
from supervision.assets import VideoAssets, download_assets
from collections import defaultdict, deque

In [ ]:
import cv2
import numpy as np
import supervision as sv
from tqdm import tqdm
from ultralytics import YOLO
from scipy.optimize import linear_sum_assignment
from collections import defaultdict, deque
import pandas as pd

SOURCE_VIDEO_PATH = '/Users/gideonowusu/Downloads/PhD 2028/Spring 2025/Coursework/5900/Trial Complete.mp4'
TARGET_VIDEO_PATH = '/Users/gideonowusu/Downloads/PhD 2028/Spring 2025/Coursework/5900/Trial Complete Out.mp4'
IOU_THRESHOLD = 0.5
MODEL_NAME = '/Users/gideonowusu/Downloads/PhD 2028/Spring 2025/Coursework/5900/Neuves (1).pt'
MODEL_RESOLUTION = 1280
TARGET_WIDTH = 25
TARGET_HEIGHT = 250
EXCEL_OUTPUT_PATH = "/Users/gideonowusu/Downloads/PhD 2028/Spring 2025/Coursework/5900/Trial Complete Trajectory Data.xlsx"
FINAL_HARDBRAKING_DATA = "/Users/gideonowusu/Downloads/PhD 2028/Spring 2025/Coursework/5900/Trial Complete Hard Braking Data.xlsx"

REAL_WORLD_WIDTH_M = 19
REAL_WORLD_HEIGHT_M = 135
PIXEL_WIDTH_PX = 25
PIXEL_HEIGHT_PX = 250
vehicle_data = []

width_ratio = REAL_WORLD_WIDTH_M / PIXEL_WIDTH_PX
height_ratio = REAL_WORLD_HEIGHT_M / PIXEL_HEIGHT_PX

initial_polygon = np.array([[270, 350], [460, 320], [1900, 660], [1720, 850]])
TARGET = np.array([[0, 0], [TARGET_WIDTH - 1, 0], [TARGET_WIDTH - 1, TARGET_HEIGHT - 1], [0, TARGET_HEIGHT - 1]])

initial_lines = [
        {"start": (340,372), "end": (374,362), "tag": "SouthEast Maj. St. Right"},
    {"start": (457,336), "end": (484,327), "tag": "SouthEast Maj. St. Left"},
    {"start": (382,358), "end": (448,340), "tag": "SouthEast Maj. St. Through"},
        
       
]
class ViewTransformer:
    def __init__(self, source, target):
        self.m = cv2.getPerspectiveTransform(source.astype(np.float32), target.astype(np.float32))

    def transform_points(self, points):
        if points.size == 0:
            return points
        reshaped_points = points.reshape(-1, 1, 2).astype(np.float32)
        transformed_points = cv2.perspectiveTransform(reshaped_points, self.m)
        return transformed_points.reshape(-1, 2)

model = YOLO(MODEL_NAME)
video_info = sv.VideoInfo.from_video_path(video_path=SOURCE_VIDEO_PATH)
frame_generator = sv.get_video_frames_generator(source_path=SOURCE_VIDEO_PATH)
byte_track = sv.ByteTrack(frame_rate=video_info.fps, track_activation_threshold=0.25, minimum_matching_threshold=0.8, lost_track_buffer=30)

bounding_box_annotator = sv.BoundingBoxAnnotator(thickness=2)
label_annotator = sv.LabelAnnotator(text_scale=0.5, text_thickness=1, text_position=sv.Position.TOP_CENTER)
trace_annotator = sv.TraceAnnotator(thickness=2, trace_length=video_info.fps * 2, position=sv.Position.BOTTOM_CENTER)

polygon_zone = sv.PolygonZone(polygon=initial_polygon)
coordinates = defaultdict(lambda: deque(maxlen=video_info.fps))
trajectory_data = []
object_tags = {}
vehicles_intersected_lines = set()
view_transformer = ViewTransformer(source=initial_polygon, target=TARGET)


# Function to create the smallest buffered bounding box
def create_smallest_buffered_box(bbox, num_steps):
    x1, y1, x2, y2 = bbox
    center_x = (x1 + x2) / 2
    center_y = (y1 + y2) / 2
    step_x = (center_x - x1) / num_steps
    step_y = (center_y - y1) / num_steps

    buffered_x1 = x1 + step_x
    buffered_y1 = y1 + step_y
    buffered_x2 = x2 - step_x
    buffered_y2 = y2 - step_y

    return (buffered_x1, buffered_y1, buffered_x2, buffered_y2)

# Function to check if line segments (p1, p2) and (q1, q2) intersect
def lines_intersect(p1, p2, q1, q2):
    def ccw(A, B, C):
        return (C[1] - A[1]) * (B[0] - A[0]) > (B[1] - A[1]) * (C[0] - A[0])

    return ccw(p1, q1, q2) != ccw(p2, q1, q2) and ccw(p1, p2, q1) != ccw(p1, p2, q2)

# Function to check if a line intersects a bounding box
def line_intersects_bbox(p1, p2, bbox):
    x1, y1, x2, y2 = bbox
    rect = [(x1, y1), (x2, y1), (x2, y2), (x1, y2)]

    for i in range(4):
        if lines_intersect(p1, p2, rect[i], rect[(i + 1) % 4]):
            return True

    return False


def callback(frame: np.ndarray, index: int) -> np.ndarray:
    global trajectory_data

    results = model(frame, verbose=False)[0]
    detections = sv.Detections.from_ultralytics(results)
    pole_detections = detections[detections.class_id == 1]
    non_pole_detections = detections[detections.class_id != 1]
    non_pole_detections = non_pole_detections[polygon_zone.trigger(non_pole_detections)]
    for line in initial_lines:
            frame = cv2.line(frame, line["start"], line["end"], (255, 0, 0), 2)
    final_detections = byte_track.update_with_detections(detections=non_pole_detections)

    # Compute vehicle positions
    for tracker_id, bbox in zip(final_detections.tracker_id, final_detections.xyxy):

        buffered_bbox = create_smallest_buffered_box(bbox, num_steps=2)

        if tracker_id not in object_tags and tracker_id is not None:
            for line in initial_lines:
                if line_intersects_bbox(line["start"], line["end"], buffered_bbox):
                            # Associate the vehicle with the first line it intersects
                    object_tags[tracker_id] = line["tag"]
                    vehicles_intersected_lines.add(tracker_id)
                    break

        if tracker_id in object_tags:
            vehicle_data.append({
                "Frame": index,
                "Center_X": (bbox[0] + bbox[2]) / 2,
                "Center_Y": (bbox[1] + bbox[3]) / 2,
                "Buffered_X1": buffered_bbox[0],
                "Buffered_Y1": buffered_bbox[1],
                "Buffered_X2": buffered_bbox[2],
                "Buffered_Y2": buffered_bbox[3],
                "Original_X1": bbox[0],
                "Original_Y1": bbox[1],
                "Original_X2": bbox[2],
                "Original_Y2": bbox[3],
                "Object ID": tracker_id,
                # "Class ID": class_id,
                "Line Tag": object_tags[tracker_id]
            })



    points = final_detections.get_anchors_coordinates(anchor=sv.Position.CENTER)
    points_transformed = view_transformer.transform_points(points=points).astype(int)
    labels = []

    for tracker_id, (x, y) in zip(final_detections.tracker_id, points_transformed):
        coordinates[tracker_id].append(y)

    for tracker_id, (x, y) in zip(final_detections.tracker_id, points_transformed):
        if len(coordinates[tracker_id]) < video_info.fps / 2:
            speed_mph = 0
        else:
            coordinate_start = coordinates[tracker_id][0]
            coordinate_end = coordinates[tracker_id][-1]

            distance_pixels = abs(coordinate_start - coordinate_end)
            distance_meters = distance_pixels * height_ratio

            time = len(coordinates[tracker_id]) / video_info.fps

            speed_m_per_s = distance_meters / time
            speed_mph = speed_m_per_s * 2.23694

        labels.append(f"#{tracker_id} {int(speed_mph)} mph")

        trajectory_data.append({


            "Frame": index,
            "Tracker ID": tracker_id,
            "Center X": x,
            "Center Y": y,
            "Speed (mph)": speed_mph,
            "Buffered_X1": buffered_bbox[0],
            "Buffered_Y1": buffered_bbox[1],
            "Buffered_X2": buffered_bbox[2],
            "Buffered_Y2": buffered_bbox[3],
            "Line Tag": object_tags.get(tracker_id, "No Tag")


        })
    cv2.polylines(frame, [initial_polygon.astype(np.int32)], isClosed=True, color=(0, 0, 255), thickness=2)

    annotated_frame = frame.copy()
    annotated_frame = trace_annotator.annotate(scene=annotated_frame, detections=final_detections)
    annotated_frame = bounding_box_annotator.annotate(scene=annotated_frame, detections=final_detections)
    annotated_frame = label_annotator.annotate(scene=annotated_frame, detections=final_detections, labels=labels)

    return annotated_frame


sv.process_video(source_path=SOURCE_VIDEO_PATH, target_path=TARGET_VIDEO_PATH, callback=callback)

trajectory_df = pd.DataFrame(trajectory_data)
trajectory_df.to_excel(EXCEL_OUTPUT_PATH, index=False)


import pandas as pd

def detect_hard_braking(input_file, output_file):
    """
    Detects hard braking incidents from an Excel file and saves the results in a new Excel file,
    including the associated Line Tag.
    """
    # Load the dataset
    df = pd.read_excel(input_file)

    hard_braking_events = []

    # Get unique vehicle IDs
    unique_vehicles = df["Tracker ID"].unique()

    for vehicle_id in unique_vehicles:
        vehicle_data = df[df["Tracker ID"] == vehicle_id].sort_values(by="Frame").reset_index(drop=True)

        for i in range(len(vehicle_data) - 19):  # Ensuring a full 20-frame window
            # Define the sliding window
            window = vehicle_data.iloc[i : i + 20]

            # Get initial and final speed
            initial_speed = window.iloc[0]["Speed (mph)"]
            final_speed = window.iloc[-1]["Speed (mph)"]

            # Get the associated Line Tag (choosing the most frequent tag in the window)
            line_tag = window["Line Tag"].mode().iloc[0] if not window["Line Tag"].mode().empty else "Unknown"

            # Compute percentage drop in speed
            if initial_speed > 0:  # Avoid division by zero
                percentage_drop = ((initial_speed - final_speed) / initial_speed) * 100

                if percentage_drop > 60 and initial_speed > 15:
                    # Check the next 100 frames for speed fluctuations
                    post_window_data = vehicle_data.iloc[i + 20 : i + 120] if i + 120 < len(vehicle_data) else vehicle_data.iloc[i + 20 :]
                    if not post_window_data.empty:
                        max_post_speed = post_window_data["Speed (mph)"].max()

                        if max_post_speed <= final_speed + 5:  # Allow slight fluctuation
                            # Check speed trend in previous 20 frames
                            pre_window_data = vehicle_data.iloc[max(i - 20, 0) : i]

                            if not pre_window_data.empty:
                                min_pre_speed = pre_window_data["Speed (mph)"].min()
                                max_pre_speed = pre_window_data["Speed (mph)"].max()

                                if min_pre_speed >= initial_speed - 5 and max_pre_speed <= initial_speed + 5:
                                    # Log hard braking event
                                    hard_braking_events.append({
                                        "Tracker ID": vehicle_id,
                                        "Frame Start": window.iloc[0]["Frame"],
                                        "Frame End": window.iloc[-1]["Frame"],
                                        "Initial Speed (mph)": initial_speed,
                                        "Final Speed (mph)": final_speed,
                                        "Percentage Drop": percentage_drop,
                                        "Max Speed in Next 100 Frames": max_post_speed,
                                        "Min Speed in Previous 20 Frames": min_pre_speed,
                                        "Max Speed in Previous 20 Frames": max_pre_speed,
                                        "Line Tag": line_tag  # Include Line Tag
                                    })

    # Convert results into a DataFrame
    hard_braking_df = pd.DataFrame(hard_braking_events)

    # Save results to an Excel file
    hard_braking_df.to_excel(output_file, index=False)

    print(f"Hard braking incidents saved to {output_file}")

detect_hard_braking(EXCEL_OUTPUT_PATH, FINAL_HARDBRAKING_DATA)

import pandas as pd

# Path to new output file for red signal timing
RED_SIGNAL_TIMING_FILE = "/Users/gideonowusu/Downloads/PhD 2028/Spring 2025/Coursework/5900/Trial Complete Red Signal Timing.xlsx"

# Define verification lines (independent from association lines)
verification_lines = {
    "SouthEast Maj. St. Left": {"start": (826,398), "end": (877,380)},
    "SouthEast Maj. St. Through": {"start": (684,450), "end": (814,410)},
}

# Read the hard braking data
braking_df = pd.read_excel(FINAL_HARDBRAKING_DATA)

# Group by 'Line Tag' and find the earliest stopping frame and corresponding Tracker ID
earliest_stopping_frames = braking_df.loc[
    braking_df.groupby("Line Tag")["Frame Start"].idxmin(),
    ["Line Tag", "Tracker ID", "Frame Start"]
].reset_index(drop=True)

earliest_stopping_frames.rename(columns={"Frame Start": "Earliest Stopping Frame"}, inplace=True)

# Function to verify red signal using verification lines
def verify_red_signal(stopping_data, trajectory_data, verification_lines):
    verification_results = []

    for _, row in stopping_data.iterrows():
        tag = row["Line Tag"]
        tracker_id = row["Tracker ID"]  # Get the Tracker ID of the first stopped vehicle
        stop_frame = row["Earliest Stopping Frame"]
        verification_line = verification_lines.get(tag)

        if not verification_line:
            continue

        # Check if any vehicle of the same tag crosses the verification line within 500 frames
        frame_end = stop_frame + 500
        movement_data = trajectory_data[
            (trajectory_data["Line Tag"] == tag) &
            (trajectory_data["Frame"] > stop_frame) &
            (trajectory_data["Frame"] <= frame_end)
        ]

        verified_red = "No" if any(
            line_intersects_bbox(verification_line["start"], verification_line["end"], bbox)
            for bbox in zip(movement_data["Buffered_X1"], movement_data["Buffered_Y1"],
                            movement_data["Buffered_X2"], movement_data["Buffered_Y2"])
        ) else "Yes"

        verification_results.append({
            "Line Tag": tag,
            "Tracker ID": tracker_id,  # Include the Tracker ID
            "Earliest Stopping Frame": stop_frame,
            "Red Signal Confirmed": verified_red
        })

    return pd.DataFrame(verification_results)

# Load trajectory data
trajectory_df = pd.read_excel(EXCEL_OUTPUT_PATH)

# Perform verification
red_signal_df = verify_red_signal(earliest_stopping_frames, trajectory_df, verification_lines)

# Save results to a new Excel file
red_signal_df.to_excel(RED_SIGNAL_TIMING_FILE, index=False)

print(f"Red signal timing verification saved to {RED_SIGNAL_TIMING_FILE}")


SupervisionWarnings: BoundingBoxAnnotator is deprecated: `BoundingBoxAnnotator` is deprecated and has been renamed to `BoxAnnotator`. `BoundingBoxAnnotator` will be removed in supervision-0.26.0.
[W NNPACK.cpp:64] Could not initialize NNPACK! Reason: Unsupported hardware.
